# ОБУЧЕНИЕ POINTNET СЕТЕЙ ПО КЛАССИФИКАЦИИ НА ОСНОВЕ ОБЛАКОВ ТОЧЕК ВЭКГ

Демонстрация нормализации исходных облаков точек:

In [7]:
import pandas as pd
import plotly.express as px

file_path = 'final_pointcloud_dataset/train/00002_period_3.csv'
file_path_normalized = 'final_pointcloud_dataset_normalized/train/00002_period_3_normalized.csv'

print('Исходное облако точек:')
# Чтение CSV-файла
df = pd.read_csv(file_path)
df['size'] = 20

# Построение 3D облака точек
fig = px.scatter_3d(df, x='x', y='y', z='z', size='size', size_max=10, opacity=1)

# Настройка внешнего вида облака точек (необязательно)
fig.update_layout(scene=dict(aspectmode='cube'))

# Отображение графика
fig.show()

print('Нормализованное облако точек:')
# Чтение CSV-файла
df_normalized = pd.read_csv(file_path_normalized)
df_normalized['size'] = 20

# Построение 3D облака точек
fig = px.scatter_3d(df_normalized, x='x', y='y', z='z', size='size', size_max=10, opacity=1)

# Настройка внешнего вида облака точек (необязательно)
fig.update_layout(scene=dict(aspectmode='cube'))

# Отображение графика
fig.show()

Исходное облако точек:


Нормализованное облако точек:
